#Mô hình học máy phân loại mã độc dựa trên tập dữ liệu về packing.

Thí nghiệm khi train với tập dữ liệu Wild với đầy đủ các nhãn được chia tỉ lệ phù hợp (tỉ lệ Benign - Malware là 50 - 50).

In [ ]:
import pandas as pd
import numpy as np
import pickle
import sklearn.ensemble as ske
from sklearn import tree, linear_model
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import SelectFromModel
import joblib
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import confusion_matrix
import random
from sklearn.metrics import classification_report

1. Đọc dữ liệu gốc từ tập Wild.

In [ ]:
df_wild = pd.read_pickle('/content/drive/MyDrive/wild.pickle')
df_wild.head()

,sample_sha1,benign,malicious,packed,unpacked_sample_sha1,unpacked_sample_id,packer_name,imp_setupremovefilelogentrya,imp_vardeccmp,imp_setdlgitemint,...,opcode_xor sub mov,opcode_xor sub mov add,opcode_xor test,opcode_xor test je,opcode_xor xor,opcode_xor xor mov,opcode_xor xor pop,opcode_xor xor pop mov,opcode_xor xor xor,opcode_xor xor xor pop
1,a2aac23e17be570c647fd22080e05b0e58449565,True,False,True,-1,-1,none,False,False,False,...,0.000022,0.0,0.000088,0.000084,0.000070,0.000066,0.000028,0.0,0.000053,0.0
2,c3aa534bbd20f3a3dd8e7a457f625291af9d0fe4,False,True,True,-1,-1,none,False,False,False,...,0.000000,0.0,0.000000,0.000000,0.000309,0.000000,0.000000,0.0,0.000331,0.0
3,bea3fb5a68ac91258e8339a8aa037304ae24242b,True,False,True,-1,-1,none,False,False,False,...,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0
10,15cb9eab6b0f0ccd718a9235f07046dff100789f,False,True,True,-1,-1,none,False,False,False,...,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0
12,8792cf08950abada1572b47948e86c9fce26c01a,True,False,True,-1,-1,none,False,False,False,...,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0


In [ ]:
print(df_wild.shape)

(50724, 56555)


2. Lọc lại 7,000 mẫu để thực nghiệm. Lọc một số cột / features không cần thiết.

In [ ]:
#Chọn ngẫu nhiên 7,000 mẫu
df = df_wild.sample(n=7000, random_state=102)

In [ ]:
#Loại bỏ một số cột không cần thiết
df = df.drop(['sample_sha1','benign','unpacked_sample_sha1','unpacked_sample_id'], axis=1)
df = df.drop(['benign_vt','malicious_vt'], axis=1)

In [ ]:
#Loại bỏ một số cột dạng string không thể xử lý
string_columns = df.select_dtypes(include=['object']).columns
df = df.drop(string_columns, axis=1)

In [ ]:
df.head()

,malicious,packed,imp_setupremovefilelogentrya,imp_vardeccmp,imp_setdlgitemint,imp_?append@?$basic_string@du?$char_traits@d@std@@v?$allocator@d@2@@std@@qaeaav12@abv12@ii@z,imp_zwwritefile,imp_glstencilop,imp_comparelinkinforeferents,imp__setmbcp,...,opcode_xor sub mov,opcode_xor sub mov add,opcode_xor test,opcode_xor test je,opcode_xor xor,opcode_xor xor mov,opcode_xor xor pop,opcode_xor xor pop mov,opcode_xor xor xor,opcode_xor xor xor pop
167256,True,True,False,False,False,False,False,False,False,False,...,0.000087,0.0,0.000329,0.000267,0.000086,0.00000,0.0,0.0,0.000000,0.0
14422,True,True,False,False,False,False,False,False,False,False,...,0.000000,0.0,0.000000,0.000000,0.000000,0.00000,0.0,0.0,0.000000,0.0
185351,True,True,False,False,False,False,False,False,False,False,...,0.000029,0.0,0.000105,0.000099,0.000085,0.00011,0.0,0.0,0.000043,0.0
53241,False,True,False,False,False,False,False,False,False,False,...,0.000000,0.0,0.000000,0.000000,0.000000,0.00000,0.0,0.0,0.000000,0.0
38618,False,True,False,False,False,False,False,False,False,False,...,0.000000,0.0,0.000199,0.000124,0.000000,0.00000,0.0,0.0,0.000000,0.0


In [ ]:
print(df.shape)

(7000, 56526)


3. Chia tập dữ liệu ra thành 5,000 mẫu để train - 2,000 mẫu để test.

In [ ]:
data = df.sample(n=5000, random_state=102)
df_2000 = df.drop(data.index)

3.1. Với tập train, chia tỉ lệ Benign - Malware (50 - 50).

In [ ]:
#Kiểm tra số lượng mẫu
UB = data[(data['malicious'] == False) & (data['packed'] == False)].shape[0]
PM = data[(data['malicious'] == True) & (data['packed'] == True)].shape[0]
PB = data[(data['malicious'] == False) & (data['packed'] == True)].shape[0]
print('Unpacked Benign: %i samples' % UB)
print('Packed Malware: %i samples' % PM)
print('Packed Benign: %i samples' % PB)
print('Total TRAIN set: %i samples'% (UB+PM+PB))

Unpacked Benign: 434 samples
Packed Malware: 3299 samples
Packed Benign: 1267 samples
Total TRAIN set: 5000 samples


In [ ]:
#Chia tỉ lệ 50 - 50
data_PM = data[(data['malicious'] == True) & (data['packed'] == True)].sample(n=(PM-(UB+PB)), random_state=102)
data = data.drop(data_PM.index)

In [ ]:
#Số lượng tập train
UB = data[(data['malicious'] == False) & (data['packed'] == False)].shape[0]
PM = data[(data['malicious'] == True) & (data['packed'] == True)].shape[0]
PB = data[(data['malicious'] == False) & (data['packed'] == True)].shape[0]
print('Unpacked Benign: %i samples' % UB)
print('Packed Malware: %i samples' % PM)
print('Packed Benign: %i samples' % PB)
print('Total TRAIN set: %i samples'% (UB+PM+PB))

Unpacked Benign: 434 samples
Packed Malware: 1701 samples
Packed Benign: 1267 samples
Total TRAIN set: 3402 samples


In [ ]:
#Xử lý tập train
y = data['malicious'].values
data = data.drop(['malicious','packed'], axis=1)

3.2. Với tập test, chia tỉ lệ Benign - Malware (50 - 50).

In [ ]:
UB = df_2000[(df_2000['malicious'] == False) & (df_2000['packed'] == False)].shape[0]
PM = df_2000[(df_2000['malicious'] == True) & (df_2000['packed'] == True)].shape[0]
PB = df_2000[(df_2000['malicious'] == False) & (df_2000['packed'] == True)].shape[0]

df_2000_PM = df_2000[(df_2000['malicious'] == True) & (df_2000['packed'] == True)].sample(n=(PM-(UB+PB)), random_state=102)
df_2000 = df_2000.drop(df_2000_PM.index)

UB = df_2000[(df_2000['malicious'] == False) & (df_2000['packed'] == False)].shape[0]
PM = df_2000[(df_2000['malicious'] == True) & (df_2000['packed'] == True)].shape[0]
PB = df_2000[(df_2000['malicious'] == False) & (df_2000['packed'] == True)].shape[0]

print('Unpacked Benign: %i samples' % UB)
print('Packed Malware: %i samples' % PM)
print('Packed Benign: %i samples' % PB)
print('Total TEST set: %i samples'% (UB+PM+PB))

Unpacked Benign: 177 samples
Packed Malware: 669 samples
Packed Benign: 492 samples
Total TEST set: 1338 samples


4. Chọn lọc ra tập các features quan trọng để dùng cho mô hình. Chia tập train - test.

In [ ]:
#Chọn lọc features
print('Researching important feature based on %i total features' % data.shape[1])
fsel = ske.ExtraTreesClassifier().fit(data, y)

Researching important feature based on 56524 total features


In [ ]:
#Tạo mô hình và chia tập train - test
model = SelectFromModel(fsel, prefit=True)

X_train = model.transform(data)
X_test = model.transform(df_2000.drop(['malicious','packed'], axis=1))
y_train = y
y_test = df_2000['malicious'].values

/usr/local/lib/python3.10/dist-packages/sklearn/base.py:432: UserWarning: X has feature names, but SelectFromModel was fitted without feature names
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/base.py:432: UserWarning: X has feature names, but SelectFromModel was fitted without feature names
  warnings.warn(


In [ ]:
#In ra số lượng features chọn được
nb_features = X_train.shape[1]
features = []
print('%i features identified as important:' % nb_features)

indices = np.argsort(fsel.feature_importances_)[::-1][:nb_features]
print(indices)

6544 features identified as important:
[42067 41699 41652 ... 55971 54052 53296]


In [ ]:
#In ra chỉ số quan trọng của từng features
for f in range(nb_features-1, -1, -1):
    print("%d. feature %s (%f)" % (f + 1, data.columns[indices[f]], fsel.feature_importances_[indices[f]]))

#Gán vào tập features để xử lý mô hình
for f in sorted(np.argsort(fsel.feature_importances_)[::-1][:nb_features]):
    features.append(data.columns[f])

Kết quả truyền trực tuyến bị cắt bớt đến 5000 dòng cuối.
5000. feature ngram_b'9d4a4a' (0.000029)
4999. feature opcode_push lea mov (0.000029)
4998. feature ngram_b'\x10\x00\x10\x00\x00\x00' (0.000029)
4997. feature ngram_b'1\x00.\x00e\x00' (0.000029)
4996. feature opcode_ret push mov sub (0.000029)
4995. feature string_b'8"u&' (0.000029)
4994. feature imp_getprofilestringw (0.000029)
4993. feature opcode_call add jmp (0.000029)
4992. feature ngram_b'  </tr' (0.000029)
4991. feature string_b'QZ^&' (0.000029)
4990. feature ngram_b'\x04\x08\x13\nWa' (0.000029)
4989. feature ngram_b'dll.dl' (0.000029)
4988. feature opcode_or push (0.000029)
4987. feature ngram_b'ft Cor' (0.000029)
4986. feature string_b'lSVW' (0.000029)
4985. feature opcode_mov push mov push (0.000029)
4984. feature opcode_pop sub (0.000029)
4983. feature opcode_jne mov mov shl (0.000029)
4982. feature string_b'.idata$7x' (0.000029)
4981. feature imp_coinitializeex (0.000029)
4980. feature string_b'OG05!' (0.000029)
4979.

5. Bắt đầu train - test.

In [ ]:
#Chọn 4 thuật toán
algorithms = {
        "DecisionTree": tree.DecisionTreeClassifier(max_depth=40),
        "RandomForest": ske.RandomForestClassifier(n_estimators=200),
        "GradientBoosting": ske.GradientBoostingClassifier(n_estimators=200),
        "AdaBoost": ske.AdaBoostClassifier(n_estimators=400)
    }
results = {}

#Train - test mô hình với lần lượt 4 thuật toán
print("Now testing algorithms")
for algo in algorithms:
    clf = algorithms[algo]
    clf.fit(X_train, y_train)
    score = clf.score(X_test, y_test)
    print("%s : %f %%" % (algo, score*100))
    results[algo] = score

Now testing algorithms
DecisionTree : 92.526158 %
RandomForest : 94.544096 %
GradientBoosting : 95.216741 %
AdaBoost : 94.917788 %


In [ ]:
#Chọn mô hình có Acc lớn nhất
winner = max(results, key=results.get)
print('Algorithm with highest accuracy on train/test is %s with a %f %% success' % (winner, results[winner]*100))

Algorithm with highest accuracy on train/test is GradientBoosting with a 95.216741 % success


6. Đánh giá kết quả.

6.1. Kết quả của mô hình có Accuracy cao nhất.

In [ ]:
#In ra các chỉ số đánh giá khác của mô hình
clf = algorithms[winner]
res = clf.predict(X_test)
mt = confusion_matrix(y_test, res)

print(winner)

print("True positive : %i " % (mt[1][1]))
print("True negative : %i " % (mt[0][0]))

print("False positive : %i " % (mt[0][1]))
print("False negative : %i " % (mt[1][0]))

GradientBoosting
True positive : 639 
True negative : 635 
False positive : 34 
False negative : 30 


In [ ]:
report = classification_report(y_test, res)
print("Classification Report:\n", report)

Classification Report:
               precision    recall  f1-score   support

       False       0.95      0.95      0.95       669
        True       0.95      0.96      0.95       669

    accuracy                           0.95      1338
   macro avg       0.95      0.95      0.95      1338
weighted avg       0.95      0.95      0.95      1338



6.2. Kết quả của mô hình RandomForest.

In [ ]:
clf = algorithms['RandomForest']
res = clf.predict(X_test)
mt = confusion_matrix(y_test, res)

print("True positive : %i " % (mt[1][1]))
print("True negative : %i " % (mt[0][0]))

print("False positive : %i " % (mt[0][1]))
print("False negative : %i " % (mt[1][0]))

True positive : 635 
True negative : 630 
False positive : 39 
False negative : 34 


In [ ]:
report = classification_report(y_test, res)
print("Classification Report:\n", report)

Classification Report:
               precision    recall  f1-score   support

       False       0.95      0.94      0.95       669
        True       0.94      0.95      0.95       669

    accuracy                           0.95      1338
   macro avg       0.95      0.95      0.95      1338
weighted avg       0.95      0.95      0.95      1338

